In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

Setting environment for Precision WorkStation
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif


No vtk
No vtkNot using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD595/MD595_prep2_thumbnail/MD595-IHC12-2015.09.15-01.21.39_MD595_2_0035_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD598/MD598_prep2_thumbnail/MD598-N18-2015.09.29-17.40.03_MD598_3_0054_prep2_thumbnail.tif
/medi

Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD658/MD658_prep2_thumbnail/MD658-N18-2017.03.31-17.34.22_MD658_2_0053_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_thumbnail/MD662&661-F30-2017.06.05-14.49.48_MD661_1_0088_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_Ntb/ChatCryoJane201710_slide1-S1_prep2_thumbnail_Ntb.tif


In [2]:
from sklearn.externals import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier 

sys.path.append('/home/yuncong/csd395/xgboost/python-package')
try:
    from xgboost.sklearn import XGBClassifier
except:
    sys.stderr.write('xgboost is not loaded.')

In [3]:
batch_size = 256
model_dir_name = 'inception-bn-blue'
model_name = 'inception-bn-blue'
model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
                                   num_gpus=1, batch_size=batch_size)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [13]:
train_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,5)'

In [28]:
stacks = ['MD661']
# test_stacks = ['MD635'] + all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks + 'ChatCryoJane201710'

for stack in stacks:

    if stack in ['MD661']:
        stack_stain = {stack: 'F'}
    else:
        stack_stain = {stack: 'N'}

    # Number of sections on which to sample examples from.
    stack_section_number = defaultdict(dict)

    for name_u in all_known_structures:
        stack_section_number[stack][name_u] = 10
    #         if name_u == '4N' or name_u == '10N':
    #             stack_section_number[st][name_u] = 20
    #         else:
    #             stack_section_number[st][name_u] = 10

    stack_section_number.default_factory = None

    
    win_id = 7
    
    grid_indices_lookup_allStacks = {}


    try:
        
        if stack in ['MD661']:        
            grid_indices_lookup_allStacks[stack] = \
            DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
                                                               by_human=False, timestamp='latest',
                                                               detector_id_f=1,
                                                              return_locations=True)            
        else:
            grid_indices_lookup_allStacks[stack] = \
            DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
                                                               by_human=True, timestamp='latest',
                                                              return_locations=True)            

    except Exception as e:
        print e
        sys.stderr.write("Fail to load annotation grid lookup for %s.\n" % stack)
        
    from itertools import chain
    all_labels = sorted(list(set(chain.from_iterable(set(grid_indices_lookup_allStacks[st].columns.tolist()) 
                                                 for st in [stack]))))
    
    def sample_addresses(stacks, structure):

        positive_labels = [structure]
        
        if stacks == ['MD661']:
            negative_labels = get_negative_labels(structure, 'neg_has_all_surround', 
                                              margin_um=200, labels_found=all_labels)
        else:
            negative_labels = get_negative_labels(structure, 'neg_has_all_surround', 
                                              margin_um=500, labels_found=all_labels)

        positive_addresses_all_stacks = {}
        negative_addresses_all_stacks = {}

        for stack in stacks:

            candidate_sections = list(chain(*[grid_indices_lookup_allStacks[stack][pl].dropna(how='any').index.tolist() 
                                          for pl in positive_labels]))

            n_sections = stack_section_number[stack][structure]

            if stack_stain[stack] == 'F':
                neurotrace_sections = []
                nissl_sections = []
                for sec in candidate_sections:
                    if metadata_cache['sections_to_filenames'][stack][sec].split('-')[1][0] == 'F':
                        neurotrace_sections.append(sec)
                    else:
                        nissl_sections.append(sec)
                sampled_sections = np.random.choice(neurotrace_sections, min(len(neurotrace_sections), n_sections), replace=False)
            else:
                sampled_sections = np.random.choice(candidate_sections, min(len(candidate_sections), n_sections), replace=False)

            positive_addresses_all_stacks[stack] = sorted([(stack, sec, tuple(loc))
    for nl in set(positive_labels) & set(grid_indices_lookup_allStacks[stack].columns)
      for sec, locs in grid_indices_lookup_allStacks[stack][nl].loc[sampled_sections].dropna().iteritems()
      for loc in locs])

            negative_addresses_all_stacks[stack] = sorted([(stack, sec, tuple(loc))
    for nl in set(negative_labels) & set(grid_indices_lookup_allStacks[stack].columns)
      for sec, locs in grid_indices_lookup_allStacks[stack][nl].loc[sampled_sections].dropna().iteritems()
      for loc in locs])

        positive_addresses = sum(positive_addresses_all_stacks.values(), [])
        negative_addresses = sum(negative_addresses_all_stacks.values(), [])

        del positive_addresses_all_stacks, negative_addresses_all_stacks

        return positive_addresses, negative_addresses
    
    
    import random

    for structure in all_known_structures:
    
        positive_addresses_traindata, negative_addresses_traindata = sample_addresses([stack], structure)

        print '# positive train =', len(positive_addresses_traindata)
        print '# negative train =', len(negative_addresses_traindata)

        positive_patches = \
    extract_patches_given_locations_multiple_sections(addresses=random.sample(positive_addresses_traindata, min(50, len(positive_addresses_traindata))),
                                                      win_id=win_id, normalization_scheme=train_scheme,
                                                      location_or_grid_index='location')

        export_fp = '/home/yuncong/training_examples/%s/%s_positive.png' % (stack, structure)
        create_parent_dir_if_not_exists(export_fp)
        display_images_in_grids(positive_patches, nc=10, cmap=plt.cm.gray, vmin=0, vmax=255, export_fn=export_fp)
        upload_to_s3(export_fp)

        ################################
        negative_patches = \
    extract_patches_given_locations_multiple_sections(addresses=random.sample(negative_addresses_traindata, min(50, len(negative_addresses_traindata))),
                                                      win_id=win_id, normalization_scheme=train_scheme,
                                                      location_or_grid_index='location')

        export_fp = '/home/yuncong/training_examples/%s/%s_negative.png' % (stack, structure)
        create_parent_dir_if_not_exists(export_fp)
        display_images_in_grids(negative_patches, nc=10, cmap=plt.cm.gray, vmin=0, vmax=255, export_fn=export_fp)
        upload_to_s3(export_fp)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD661" "/home/yuncong/CSHL_labelings_v3/MD661" --exclude "*" --include "*win7*warp*grid_indices_lookup*"
latest timestamp:  01302018020813
# positive train = 1028
# negative train = 5726


No images are provided. Load instead.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_lossless/MD662&661-F104-2017.06.06-23.26.43_MD661_1_0310_prep2_lossless.tif


Load image: 23.82 seconds.
Crop patches: 0.00 seconds.
No images are provided. Load instead.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_lossless/MD662&661-F110-2017.06.07-02.03.05_MD661_1_0328_prep2_lossless.tif


...

/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_lossless/MD662&661-F62-2017.06.06-04.35.55_MD661_1_0184_prep2_lossless.tif


Load image: 15.27 seconds.
Crop patches: 0.00 seconds.
No images are provided. Load instead.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_lossless/MD662&661-F64-2017.06.06-05.35.10_MD661_1_0190_prep2_lossless.tif


Load image: 15.75 seconds.
Crop patches: 0.00 seconds.
No images are provided. Load instead.
Not using image_cache.


KeyboardInterrupt: 